<a href="https://colab.research.google.com/github/Papa-Panda/Paper_reading/blob/main/toy_self_driving.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# toy example of self-driving
# next steps: BEV net, occupancy net, end to end

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

# Perception: Object detection network (simplified CNN)
class PerceptionNet(nn.Module):
    def __init__(self):
        super(PerceptionNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.fc = nn.Linear(32 * 8 * 8, 4)  # Output: bounding box [x, y, w, h]

    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

# Prediction: Trajectory prediction network (RNN)
# class PredictionNet(nn.Module):
#     def __init__(self):
#         super(PredictionNet, self).__init__()
#         self.rnn = nn.LSTM(input_size=4, hidden_size=16, num_layers=1, batch_first=True)
#         self.fc = nn.Linear(16, 2)  # Output: predicted position [dx, dy]

#     def forward(self, x):
#         x, _ = self.rnn(x)
#         return self.fc(x[:, -1, :])
class PredictionNet(nn.Module):
    def __init__(self):
        super(PredictionNet, self).__init__()
        self.rnn = nn.LSTM(input_size=8, hidden_size=16, num_layers=1, batch_first=True)  # Update input_size to 8
        self.fc = nn.Linear(16, 2)  # Output: predicted position [dx, dy]

    def forward(self, x):
        x, _ = self.rnn(x)
        return self.fc(x[:, -1, :])


# Planning: Path planning network (MLP)
class PlanningNet(nn.Module):
    def __init__(self):
        super(PlanningNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(8, 32),  # Adjust input size to match actual input shape
            nn.ReLU(),
            nn.Linear(32, 2)  # Output: planned movement [dx, dy]
        )

    def forward(self, x):
        return self.fc(x)


# Simulated data and pipeline
batch_size = 4
image_data = torch.randn(batch_size, 3, 32, 32)  # Example images
past_trajectory = torch.randn(batch_size, 5, 4)  # Example past trajectories
car_state = torch.randn(batch_size, 2)  # Example car state [x, y]

def train_self_driving():
    perception_net = PerceptionNet()
    prediction_net = PredictionNet()
    planning_net = PlanningNet()

    criterion = nn.MSELoss()
    optimizer = optim.Adam(
        list(perception_net.parameters()) +
        list(prediction_net.parameters()) +
        list(planning_net.parameters()), lr=0.001
    )

    for epoch in range(10):  # Training loop
        # Perception: Detect objects
        detected_boxes = perception_net(image_data)  # Outputs bounding boxes [batch_size, 4]

        # Prediction: Predict future trajectory
        prediction_input = torch.cat((past_trajectory, detected_boxes.unsqueeze(1).repeat(1, 5, 1)), dim=2)
        predicted_trajectory = prediction_net(prediction_input)

        # Planning: Generate path to avoid collision
        planning_input = torch.cat((car_state, predicted_trajectory, detected_boxes), dim=1)
        planned_path = planning_net(planning_input)

        # Compute loss (dummy target for illustration)
        target_path = torch.zeros_like(planned_path)  # Assume the goal is [0, 0]
        loss = criterion(planned_path, target_path)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch + 1}, Loss: {loss.item():.4f}")

train_self_driving()


Epoch 1, Loss: 0.0659
Epoch 2, Loss: 0.0352
Epoch 3, Loss: 0.0183
Epoch 4, Loss: 0.0082
Epoch 5, Loss: 0.0055
Epoch 6, Loss: 0.0033
Epoch 7, Loss: 0.0022
Epoch 8, Loss: 0.0010
Epoch 9, Loss: 0.0016
Epoch 10, Loss: 0.0022
